# CS231n Proj Codes
+ CNN from CS231n Proj : "Facial keypoints detection using Neural Network"

## 1. Load the Dataset

In [1]:
import os
import numpy as np
import pandas as pd
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
from collections import OrderedDict
from matplotlib import pyplot

In [2]:
## Load the dataset in 1-D form
def load(file_name, test=False, cols=None):
    df = pd.read_csv(file_name)

    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))

    if cols: # choose given row
        df = df[list(cols) + ['Image']]

    # print(df.count()) Basic count of different features
    df = df.dropna() # We can see that some values are missing, we just use those completed data

    X = np.vstack(df['Image'].values) / 255. # Normalization
    X = X.astype(np.float32)

    if not test:
        y = df[df.columns[:-1]].values
        y = (y - 48) / 48
        X, y = shuffle(X, y, random_state=42)
        y = y.astype(np.float32)
    else:
        y = None

    return X, y

In [3]:
# Load the dataset in 2-D form
def load2d(file_name, test=False, cols=None):
    X, y = load(file_name, test, cols)
    X = X.reshape(-1, 96, 96, 1) # if theano backend --> -1 , 1, 96, 96
    return X, y

In [4]:
X1d, y1d = load('training.csv')
print(X1d.shape)
print(y1d.shape)

(2140, 9216)
(2140, 30)


In [21]:
X2d, y2d = load2d('training.csv')
print(X2d.shape)
print(y2d.shape)

(2140, 96, 96, 1)
(2140, 30)


## 2. Models:
+ model_hidden : one hidden layer neural network
+ model_cnn : 5-layer CNN
+ Inception : Use GoogLeNet to extract features first
    + model_inception_hidden : Inception + one hidden layer neural network
    + model_inception_cnn : Inception + 5-layer CNN
    + model_inception_dropout : Add dropout layer in the cnn model as a third Inception model to overcome overfitting

## 3. Hidden

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

Using TensorFlow backend.


In [10]:
model_hidden = Sequential() 
model_hidden.add(Dense(100, input_dim = 9216)) # Input the 1-D data
model_hidden.add(Activation("relu"))
model_hidden.add(Dense(30))
sgd = SGD(lr = 0.01, momentum = 0.9, nesterov = True)
model_hidden.compile(loss = 'mean_squared_error', optimizer = sgd, metrics=['mae', 'acc'])

In [18]:
## Build CV dataset
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5)
for train_index, test_index in kf.split(X1d):
    X1d_train, X1d_cv = X1d[train_index], X1d[test_index]
    y1d_train, y1d_cv = y1d[train_index], y1d[test_index]
print(X1d_train.shape)
print(X1d_cv.shape)

(1712, 9216)
(428, 9216)


In [17]:
model_hidden.fit(X1d_train, y1d_train, nb_epoch = 400, batch_size = 1, validation_split = 0.2)

(1712, 9216)
(428, 9216)


In [ ]:
## Predict and evaluate the result
cv_prediction = model_hidden.predict(X1d_test)


In [19]:
## Get the prediction of test dataset


In [ ]:
## Save the model and test results

## 4. CNN

In [20]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten
from keras.optimizers import SGD 
# Other choices : RMSprop / Adam (RMSprop with momentem) / Adamax / Nadam (Nesterov version)

In [ ]:
model_cnn = Sequential()

# Filters : 32
# Kernel size : 3 * 3
# Input shape 96 * 96 * 1
model_cnn.add(Convolution2D(32, 3, 3, input_shape=(96, 96, 1)))
model_cnn.add(Activation('relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Convolution2D(64, 2, 2))
model_cnn.add(Activation('relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Convolution2D(128, 2, 2))
model_cnn.add(Activation('relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Flatten()) # Flatten the multi-dimentional input into 1-D for Dense layer
model_cnn.add(Dense(500))
model_cnn.add(Activation('relu'))
model_cnn.add(Dense(500))
model_cnn.add(Activation('relu'))
model_cnn.add(Dense(30))

In [ ]:
sgd = SGD(lr=0.01, momentum = 0.9, nesterov = True)
model_cnn.compile(loss = 'mean_squared_error', optimizer = sgd, metrics=['acc'])
model_cnn.fit(X2d, y2d, nb_epoch = 400, validation_split = 0.2)